In [0]:
import os
import pandas as pd
import numpy as np
import timeit
import ujson
#Usamos ujson que parece que es el que hace la importación más rápida. Añadir la documentación donde se comenta.
#from google.colab import drive  #Lo pongo más abajo, porque si no me da error (por qué??!!)

#Para conocer el uso de la memoria
import os 
import psutil

In [2]:
!ls -lh

total 3.2G
-rw-r--r-- 1 root root 2.5K Oct 28 10:31 adc.json
-rw-r--r-- 1 root root 2.7M Oct 28 11:31 Cast_Crew_ids.csv
drwx------ 3 root root 4.0K Oct 28 10:34 drive
-rw-r--r-- 1 root root 1.5M Oct 28 10:41 IMDB_5000_movie.csv
-rw-r--r-- 1 root root 124K Oct 28 11:24 Movies_ids.csv
-rw-r--r-- 1 root root 1.5G Oct 28 10:32 movies.zip
-rw-r--r-- 1 root root 1.8G Oct 28 11:31 people.zip
drwxr-xr-x 2 root root 4.0K Oct 25 17:24 sample_data
drwxr-xr-x 4 root root 4.0K Oct 28 11:33 themoviedb_data


##IMPORTACIÓN DESDE GOOGLE DRIVE

In [0]:
#Esto sólo funciona en Google Colab porque tiene una librería de Google Colab.
#Más documentación en:
#https://colab.research.google.com/notebooks/io.ipynb
#en el apartado PyDrive.
#PyDrive documentación:
#https://gsuitedevs.github.io/PyDrive/docs/build/html/index.html

# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz

In [0]:
#Estos ficheros están en mi google drive.

#Vínculo a la carpeta comprimida con todas las pelis en mi Google Drive:
#https://drive.google.com/open?id=1-jrX_p1VfgMceGIj7SK_eIef8syIGsdL

#people data
file_id = '1UnpPTqVcghfQzj55RIoLm8pBH_Rjdl4f'

downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("people.zip")

#people ids
file_id = '19oOa9qjMFnDv19DHmNTTJKIuGW8X5BkQ'

downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("Cast_Crew_ids.csv")

In [6]:
!ls -lh

total 3.2G
-rw-r--r-- 1 root root 2.5K Oct 28 10:31 adc.json
-rw-r--r-- 1 root root 2.7M Oct 28 13:38 Cast_Crew_ids.csv
drwx------ 3 root root 4.0K Oct 28 10:34 drive
-rw-r--r-- 1 root root 1.5M Oct 28 10:41 IMDB_5000_movie.csv
-rw-r--r-- 1 root root 124K Oct 28 11:24 Movies_ids.csv
-rw-r--r-- 1 root root 1.5G Oct 28 10:32 movies.zip
-rw-r--r-- 1 root root 1.8G Oct 28 13:37 people.zip
drwxr-xr-x 2 root root 4.0K Oct 25 17:24 sample_data
drwxr-xr-x 4 root root 4.0K Oct 28 11:33 themoviedb_data


In [7]:
#Da un error pero no tiene consecuencias

#OJO!!!
#Si he ejecutado antes las pelis, no tengo que crear otra vez este directorio:
#!mkdir themoviedb_data

!cd themoviedb_data && unzip -q ../people.zip
!ls -hl themoviedb_data

warning [../people.zip]:  76 extra bytes at beginning or within zipfile
  (attempting to process anyway)
error [../people.zip]:  reported length of central directory is
  -76 bytes too long (Atari STZip zipfile?  J.H.Holm ZIPSPLIT 1.1
  zipfile?).  Compensating...
error:  expected central file header signature not found (file #1197557).
  (please check that you have transferred or created the zipfile in the
  appropriate BINARY mode and that you have compiled UnZip properly)
total 45M
drwxr-xr-x 2 root root 12M Oct 28 10:33 movies
drwxr-xr-x 2 root root 33M Oct 28 11:36 people


In [8]:
!ls -hl ./themoviedb_data/people | head -10

total 6.6G
-rw-r--r-- 1 root root  15K Sep  2 05:00 1000007.json
-rw-r--r-- 1 root root 2.8K Aug 30 06:49 100000.json
-rw-r--r-- 1 root root 1.4K Aug 30 06:49 100001.json
-rw-r--r-- 1 root root 1.4K Aug 30 06:49 100002.json
-rw-r--r-- 1 root root 3.1K Sep  2 05:00 1000034.json
-rw-r--r-- 1 root root  16K Aug 30 06:49 100003.json
-rw-r--r-- 1 root root  841 Aug 30 06:49 100004.json
-rw-r--r-- 1 root root 3.9K Sep  2 05:00 1000057.json
-rw-r--r-- 1 root root 3.1K Sep  2 05:01 1000059.json


In [7]:
!ls -1 themoviedb_data/people | wc -l

1197556


In [8]:
#Para saber el uso de memoria que estoy haciendo:
process = psutil.Process(os.getpid())
print(process.memory_info().rss)

3825057792


In [0]:
#Aunque haga lo siguiente, no libero la memoria (eliminando un set sólo con %xdel he visto que sí, pero no un dataframe)
#dfPeopleId = pd.DataFrame()
#%xdel dfPeopleIds

In [66]:
%whos

Variable            Type               Data/Info
------------------------------------------------
GoogleAuth          type               <class 'pydrive.auth.GoogleAuth'>
GoogleCredentials   type               <class 'oauth2client.client.GoogleCredentials'>
GoogleDrive         type               <class 'pydrive.drive.GoogleDrive'>
a                   list               n=0
auth                module             <module 'google.colab.aut<...>es/google/colab/auth.py'>
columnName          str                movie_credits_cast
data_iterator       function           <function data_iterator at 0x7f0a7b3a9f28>
dfPeople            DataFrame                   adult    birthda<...>203159 rows x 13 columns]
dfPeopleIds         DataFrame                  Unnamed: 0       <...>[203181 rows x 2 columns]
downloaded          GoogleDriveFile    GoogleDriveFile({'id': '1<...>Q', 'spaces': ['drive']})
drive               GoogleDrive        <pydrive.drive.GoogleDriv<...>object at 0x7f0b9167b1d0>
file_id  

##ACCESO A MI DRIVE

In [18]:
#Esto me va a permitir tener acceso a las carpetas de mi drive desde Colab:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
!ls "/content/drive/My Drive/" -lh

total 3.4G
drwx------ 2 root root 4.0K Jul 14 07:08  20180714_Deep_Learning
-rw------- 1 root root  76K Oct 22 18:13  6885.json
drwx------ 2 root root 4.0K Jun 26  2016 'avast! Files (XT1068, UWbYzWPVmAmwlrlD)'
-rw------- 1 root root 2.7M Oct 28 11:25  Cast_Crew_ids.csv
drwx------ 2 root root 4.0K Jul 18 18:38 'Colab Notebooks'
-rw------- 1 root root  44K Feb  4  2018  contacts.vcf
-rw------- 1 root root 170M Oct 22 12:05  dfMoviesCast.csv
-rw------- 1 root root    1 Apr  9  2018  facturaIVA.gsheet
-rw------- 1 root root 1.5M Oct 24 17:34  IMDB_5000_movie.csv
-rw------- 1 root root    1 Oct 28 09:54 'Mis cuentas.gsheet'
-rw------- 1 root root 124K Oct 28 11:25  Movies_ids.csv
-rw------- 1 root root 1.5G Oct 12 15:09  movies.zip
-rw------- 1 root root    1 Oct 20 08:40 'Nuestras Cuentas.gsheet'
-rw------- 1 root root 1.8G Oct 27 13:06  people.zip
-rw------- 1 root root    1 Jul 24 18:17 'Propuesta de una idea.gslides'
drwx------ 2 root root 4.0K Apr  7  2015  Screencastify
-rw------- 1 

## CONTROLANDO ESTRUCTURA Y FORMATOS ANTES DE IMPORTAR LOS DATOS A UN DATAFRAME

In [0]:
#import os
#import pandas as pd
#import timeit
#import ujson
#Usamos ujson que parece que es el que hace la importación más rápida. Añadir la documentación donde se comenta.
#import numpy as np #lo uso más adelante

path = "./themoviedb_data/people"

In [0]:
#El código de abajo sigue realmente petando si le pido que importe un múltiplo de 1000 ficheros. Arreglar/eliminar este punto !!!!!!!!!!

In [23]:
tic=timeit.default_timer()

dfTypeCheck = pd.DataFrame()
dfAux = pd.DataFrame()
intI = 0

for root, dirs, files in os.walk(path):
    for f in files:
        intI += 1
        if intI <= 2000000:
            #Si el valor aquí es un múltiplo de 5000, la celda me devolverá un error. Porque dfAux quedará en blanco para la instrucción tras el "for".
            #Está finalmente resuelto.
            if f.endswith('.json'):
                fp = os.path.join(root,f)
                with open(fp) as o:
                
                    data = ujson.load(o)
                    dfAux = dfAux.append(pd.DataFrame([(x, 'NotDict', str(type(data[x])), 'aux') for x in data.keys() if not isinstance(data[x], dict)], 
                                                              columns = ['key1', 'key2', 'type', 'aux']))
                    dfAux = dfAux.append(pd.DataFrame([(x, y, str(type(data[x][y])), 'aux') for x in data.keys() if isinstance(data[x], dict) 
                                                   for y in data[x].keys()], columns = ['key1', 'key2', 'type', 'aux']))
                    if intI % 1000 == 0:
                        dfTypeCheck = dfTypeCheck.append(dfAux.groupby(['key1','key2','type']).count())
                        dfAux = pd.DataFrame()
                        print(intI)
        else:
            break
            
if intI % 1000 != 0:
    dfTypeCheck = dfTypeCheck.append(dfAux.groupby(['key1','key2','type']).count())
    dfAux = pd.DataFrame()

dfTypeCheck = dfTypeCheck.groupby(['key1','key2','type']).sum()

toc=timeit.default_timer()
toc - tic #elapsed time in seconds

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

5294.494920729001

In [24]:
process = psutil.Process(os.getpid())
print(process.memory_info().rss)

2078261248


In [26]:
intI

1197556

In [27]:
dfTypeCheck.shape

(54, 1)

In [28]:
dfTypeCheck

aux
key1                 key2         type                       
adult                NotDict      <class 'bool'>      1197556
also_known_as        NotDict      <class 'list'>      1197556
biography            NotDict      <class 'str'>       1197556
birthday             NotDict      <class 'NoneType'>  1082735
                                  <class 'str'>        114821
deathday             NotDict      <class 'NoneType'>  1166166
                                  <class 'str'>         31390
external_ids         cast         <class 'list'>            2
                     crew         <class 'list'>            2
                     facebook_id  <class 'NoneType'>  1190356
                                  <class 'str'>          7197
                     freebase_id  <class 'NoneType'>  1151955
                                  <class 'str'>         45599
                     freebase_mid <class 'NoneType'>  1125377
                                  <class 'str'>         72177
                     imdb_id      <class 'NoneType'>   635960
                                  <class 'str'>        561591
                     instagram_id <class 'NoneType'>  1190081
                                  <class 'str'>          7473
                     tvrage_id    <class 'NoneType'>  1156373
                                  <class 'int'>         40902
                     twitter_id   <class 'NoneType'>  1186610
                                  <class 'str'>         10943
gender               NotDict      <class 'int'>       1197556
homepage             NotDict      <class 'NoneType'>  1184167
                                  <class 'str'>         13389
id                   NotDict      <class 'int'>       1197556
images               facebook_id  <class 'NoneType'>        2
                     freebase_id  <class 'NoneType'>        1
                                  <class 'str'>             1
                     freebase_mid <class 'NoneType'>        1
                                  <class 'str'>             1
                     imdb_id      <class 'NoneType'>        1
                                  <class 'str'>             1
                     instagram_id <class 'NoneType'>        2
                     profiles     <class 'list'>      1197554
                     tvrage_id    <class 'NoneType'>        1
                                  <class 'int'>             1
                     twitter_id   <class 'NoneType'>        2
imdb_id              NotDict      <class 'str'>       1197553
known_for_department NotDict      <class 'NoneType'>      418
                                  <class 'str'>       1197138
movie_credits        NotDict      <class 'list'>            2
                     cast         <class 'list'>      1197554
                     crew         <class 'list'>      1197554
name                 NotDict      <class 'str'>       1197556
place_of_birth       NotDict      <class 'NoneType'>  1108933
                                  <class 'str'>         88623
popularity           NotDict      <class 'float'>      478769
                                  <class 'int'>        718787
profile_path         NotDict      <class 'NoneType'>  1038845
                                  <class 'str'>        158711
tv_credits           cast         <class 'list'>      1197556
                     crew         <class 'list'>      1197556

In [0]:
#EXPORTO A UN CSV
dfTypeCheck.to_csv('typesCheckPeople.csv')

In [30]:
#PARA EXPORTAR A EXCEL 
!pip install openpyxl
import openpyxl

dfTypeCheck.to_excel('typesCheckPeople.xlsx')

    100% |████████████████████████████████| 1.9MB 11.1MB/s 
  Running setup.py bdist_wheel for openpyxl ... - \ | done
  Stored in directory: /root/.cache/pip/wheels/57/41/b9/3765af8bda4a8d4b6aaf4957d7214984c3332348713e85cf36
  Running setup.py bdist_wheel for et-xmlfile ... - done
  Stored in directory: /root/.cache/pip/wheels/2a/77/35/0da0965a057698121fc7d8c5a7a9955cdbfb3cc4e2423cad39
Successfully built openpyxl et-xmlfile


In [0]:
#Copio ambos ficheros de Colab a mi drive. Y después, desde Drive, ya los puedo bajar a local.
!cp typesCheckPeople.* /content/drive/"My Drive"

In [33]:
!ls -lh "/content/drive/My Drive/"

total 3.4G
drwx------ 2 root root 4.0K Jul 14 07:08  20180714_Deep_Learning
-rw------- 1 root root  76K Oct 22 18:13  6885.json
drwx------ 2 root root 4.0K Jun 26  2016 'avast! Files (XT1068, UWbYzWPVmAmwlrlD)'
drwx------ 2 root root 4.0K Jul 18 18:38 'Colab Notebooks'
-rw------- 1 root root  44K Feb  4  2018  contacts.vcf
-rw------- 1 root root 170M Oct 22 12:05  dfMoviesCast.csv
-rw------- 1 root root    1 Apr  9  2018  facturaIVA.gsheet
-rw------- 1 root root 1.5M Oct 24 17:34  IMDB_5000_movie.csv
-rw------- 1 root root    1 Oct 27 14:59 'Mis cuentas.gsheet'
-rw------- 1 root root 1.5G Oct 12 15:09  movies.zip
-rw------- 1 root root    1 Oct 20 08:40 'Nuestras Cuentas.gsheet'
-rw------- 1 root root 1.8G Oct 27 13:06  people.zip
-rw------- 1 root root    1 Jul 24 18:17 'Propuesta de una idea.gslides'
drwx------ 2 root root 4.0K Apr  7  2015  Screencastify
-rw------- 1 root root    1 Sep 30  2017  Series.gdoc
-rw------- 1 root root    1 Jan  2  2016 'Trabajos Iñigo.gsheet'
-rw-------

## IMPORTACIÓN DE LOS JSON'S DE PELÍCULAS A UN DATAFRAME

### PRIMEROS ANÁLISIS PARA DESCARTAR CAMPOS

In [42]:
#Importo aquí la mayoría de los campos (sólo excluyo los que ya sé con seguridad que no voy a usar.)
#Después extraeré unos 1000 registros para analizar si todavía voy a quitar más campos.
#En la especificación de este código ha sido clave la visión que nos da dfCheckTypes.

#Usamos un generator, que es más eficiente en cuanto a memoria. "yield" va asociado al generator.

path = "./themoviedb_data/people"
dfPeople = pd.DataFrame()

def data_iterator(path):
    intI = 0
    for root, dirs, files in os.walk(path):
        for f in files[0:1000]:
            intI += 1
            if f.endswith('.json'):
                fp = os.path.join(root,f)
                with open(fp) as o:
                    data = ujson.load(o)
                
                if intI % 10000 == 0:
                    print(intI)
                    
                result = {"adult": data["adult"], 
                       "also_known_as": data["also_known_as"], "biography": data["biography"],
                       "birthday": data["birthday"], "deathday": data["deathday"], 
                       "external_ids_imdb_id": data["external_ids"].get("imdb_id",None),
                       "gender": data["gender"], "homepage": data["homepage"], "id": data["id"],
                       "images_profiles": data["images"].get("profiles",None), "imdb_id": data.get("imdb_id",None),
                       "known_for_department": data["known_for_department"], "name": data["name"], "place_of_birth": data["place_of_birth"],
                       "popularity": data["popularity"], "profile_path": data["profile_path"],
                       "tv_credits_cast": data["tv_credits"]["cast"], "tv_credits_crew": data["tv_credits"]["crew"]}

                #After the dfTypeCheck, we know that movie_credits is not a dictionary in two rows (probably this is an error.). 
                #So we need to take this situation into consideration:
                if isinstance(data["movie_credits"], dict):
                    result["movie_credits_cast"] = data["movie_credits"]["cast"]
                    result["movie_credits_crew"] = data["movie_credits"]["crew"]
                else:
                    result["movie_credits_cast"] = None
                    result["movie_credits_crew"] = None

                yield result  
                  
tic=timeit.default_timer()
dfPeople = pd.DataFrame(data_iterator(path))
dfPeople.set_index('id', inplace=True)
toc=timeit.default_timer()
toc - tic #elapsed time in seconds


1.5791181559980032

In [43]:
dfPeople.shape

(1000, 19)

In [44]:
#Imagino que podría excluir los adult=True, pero los voy a dejar, porque: ¿cómo gestionan si un actor ahora no lo es pero lo fue en el pasado?
dfPeople['adult'].value_counts()

False    987
True      13
Name: adult, dtype: int64

In [54]:
dfPeople[dfPeople['also_known_as'].map(len)!=0][['name', 'also_known_as']]
#Me da un poco igual este campo (also_known_as). No lo tomaré.

,name,also_known_as
id,,
1326619,Carol Abboud,[Carole Abboud]
1782356,Stella Hacta,[Stella]
1391700,Drew Davidson,[Andrew Davidson]
1144677,Dionysis Xanthos,[Διονύσης Ξανθός ]
61011,Clarke Peters,[Peter Clarke]
1778600,Miyuki Nagato,[長門美由樹]
115395,Aleksey Petrenko,"[Aleksei Petrenko, Алексей Петренко, Aleksey P..."
140294,Yoo Tae-woong,"[유태웅, Yu Tae-woong]"
18300,Eddie J. Fernandez,"[Eddie Fernandez, Edward J. Fernandez , Edward..."


In [59]:
#Tampoco voy a usar este campo.
dfPeople[dfPeople['biography'].str.len()>0]['biography'].head(10)

id
61011      ​From Wikipedia, the free encyclopedia.\n\nCla...
115395     Ukrainian native Alexei Vasilyevich Petrenko (...
140294          Yoo Tae-woong (유태웅) is a South Korean actor.
1049354    Ivan Zarić is a Serbian actor. Son of actor Dr...
141043     Samuel B. "Sam" Register (born June 16, 1969) ...
1477275    Robert Carl Cohen is a director and writer, kn...
173810     Egyptian-born film-maker and actor Sayed Badre...
92564      From Wikipedia, the free encyclopedia\n\nEldon...
99727      ​From Wikipedia, the free encyclopedia.  \n\nM...
1559370    Cassandra Frechette works as a script supervis...
Name: biography, dtype: object

In [63]:
dfPeople[['external_ids_imdb_id', 'imdb_id']].head(10)
#Voy a tomar sólo imdb_id

,external_ids_imdb_id,imdb_id
id,,
1171415,nm4364562,nm4364562
1645780,None,
2079883,None,
1457505,nm1068372,nm1068372
1883146,None,
1326619,nm0008084,nm0008084
1512694,nm2468421,nm2468421
1782356,nm2393055,nm2393055
2110768,None,


In [65]:
#Tampoco voy a usar este:
dfPeople[dfPeople['homepage'].notnull()]['homepage'].head()

id
173810               http://www.sayedbadreya.com/
2030011               https://www.hailiesahar.com
1047300            http://www.sergiobarrejon.com/
86616      https://fredericgilles.com/bio-english
1358251           http://www.markusrothkranz.com/
Name: homepage, dtype: object

In [67]:
dfPeople[dfPeople['images_profiles'].map(len)!=0]['images_profiles'].head()

id
1326619    [{'iso_639_1': None, 'aspect_ratio': 0.6666666...
61782      [{'iso_639_1': None, 'aspect_ratio': 0.66625, ...
134803     [{'iso_639_1': None, 'aspect_ratio': 0.80625, ...
237408     [{'iso_639_1': None, 'aspect_ratio': 0.6666666...
1773875    [{'iso_639_1': None, 'aspect_ratio': 0.6666666...
Name: images_profiles, dtype: object

In [68]:
#Tampoco voy a usar images_profiles:
dfPeople[dfPeople['images_profiles'].map(len)!=0]['images_profiles'].iloc[0]

[{'aspect_ratio': 0.66666666666667,
  'file_path': '/3C0Px1NxXpA5VPeQNr04uJB2y4p.jpg',
  'height': 450,
  'iso_639_1': None,
  'vote_average': 0,
  'vote_count': 0,
  'width': 300}]

In [70]:
#No creo que lo use, pero lo dejaré:
dfPeople['known_for_department'].head()

id
1171415       Acting
1645780        Sound
2079883       Acting
1457505    Directing
1883146        Sound
Name: known_for_department, dtype: object

In [73]:
#Lo voy a dejar de momento, por si me vale para crear una variable "actor extranjero". Pero veo que está muy poco informado.
dfPeople[dfPeople['place_of_birth'].notnull()]['place_of_birth'].head()

id
1773875          Seattle, Washington, USA
61011      New York City - New York - USA
1724699                      Ulm, Germany
228011           Morrinhos, Goiás, Brazil
1049354                  Belgrade, Serbia
Name: place_of_birth, dtype: object

In [75]:
#No la tomo:
dfPeople[dfPeople['profile_path'].notnull()]['profile_path'].head()

id
1326619    /3C0Px1NxXpA5VPeQNr04uJB2y4p.jpg
61782      /hYyo2lWLZBzmelUOnhz3szpd9Lf.jpg
134803     /pbSHZyKBOEABayw8F5XsDnW5Qcp.jpg
237408     /hpZwHhZcGCBrl3irmrRAZxwEQca.jpg
1773875    /jj9SitqPcDIGwUGe93hValmJOED.jpg
Name: profile_path, dtype: object

In [76]:
dfPeople['movie_credits_cast'].head()

id
1171415    [{'release_date': '2013-03-22', 'adult': False...
1645780                                                   []
2079883    [{'poster_path': None, 'adult': False, 'backdr...
1457505    [{'release_date': '2011-11-23', 'adult': False...
1883146                                                   []
Name: movie_credits_cast, dtype: object

In [79]:
#Entiendo que esto será una lista de diccionarios, con un diccionario para cada peli.
dfPeople['movie_credits_cast'].iloc[0]

[{'adult': False,
  'backdrop_path': None,
  'character': 'Herself',
  'credit_id': '5b40eb009251414f36006703',
  'genre_ids': [],
  'id': 534437,
  'original_language': 'en',
  'original_title': 'Nellie Bly Makes the News',
  'overview': 'An animated documentary about the legendary journalist who changed the game for women in reporting before women even had the right to vote. Examining boundaries between reporting and storytelling, it creates a dynamic portrait of a woman who refused to accept the status quo.',
  'popularity': 0.157,
  'poster_path': None,
  'release_date': '2018-04-18',
  'title': 'Nellie Bly Makes the News',
  'video': False,
  'vote_average': 0,
  'vote_count': 0}]

In [0]:
#Ya sé qué campos tomar (en cuatro de ellos todavía tendré que aplicar cambios de formato.)

### IMPORTO LAS PELÍCULAS A UN DATAFRAME, YA EXCLUYENDO ALGUNOS CAMPOS Y SÓLO TOMANDO LOS ID'S QUE HAN TRBAJADO EN PELIS CON REVENUE_FINAL>0

In [10]:
#Reading file and saving in df
dfPeopleIds = pd.read_csv('./Cast_Crew_ids.csv')
dfPeopleIds.tail()

,Unnamed: 0,id
203176,203176,1112938
203177,203177,1112939
203178,203178,1112940
203179,203179,34438
203180,203180,567605


In [0]:
setPeopleIds = set(dfPeopleIds['id'])

In [12]:
#Usamos un generator, que es más eficiente en cuanto a memoria. "yield" va asociado al generator.
#Excluyo: also_known_as, biography, external_ids_imdb_id, homepage, images_profiles, profile_path
#Además, sólo tomo los ids que están en el set setPeopleIds

path = "./themoviedb_data/people"
dfPeople = pd.DataFrame()

def data_iterator(path):
    intI = 0
    for root, dirs, files in os.walk(path):
        for f in files:
            intI += 1
            if f.endswith('.json'):
                fp = os.path.join(root,f)
                with open(fp) as o:
                    data = ujson.load(o)
                
                if intI % 10000 == 0:
                    print(intI)
                    
                result = {"adult": data["adult"], 
                          "birthday": data["birthday"], "deathday": data["deathday"], 
                          "gender": data["gender"], "id": data["id"], "imdb_id": data.get("imdb_id",None),
                          "known_for_department": data["known_for_department"], "name": data["name"], "place_of_birth": data["place_of_birth"],
                          "popularity": data["popularity"], "tv_credits_cast": data["tv_credits"]["cast"], 
                          "tv_credits_crew": data["tv_credits"]["crew"]}

                #After the dfTypeCheck, we know that movie_credits is not a dictionary in two rows (probably this is an error.). 
                #So we need to take this situation into consideration:
                if isinstance(data["movie_credits"], dict):
                    result["movie_credits_cast"] = data["movie_credits"]["cast"]
                    result["movie_credits_crew"] = data["movie_credits"]["crew"]
                else:
                    #Aquí es importante poner una lista de longitud vacía, que nos va a evitar problemas.
                    result["movie_credits_cast"] = list()
                    result["movie_credits_crew"] = list()

                if data["id"] in setPeopleIds:    
                    yield result  
                  
tic=timeit.default_timer()
dfPeople = pd.DataFrame(data_iterator(path))
dfPeople.set_index('id', inplace=True)
toc=timeit.default_timer()
toc - tic #elapsed time in seconds


10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000
790000
800000
810000
820000
830000
840000
850000
860000
870000
880000
890000
900000
910000
920000
930000
940000
950000
960000
970000
980000
990000
1000000
1010000
1020000
1030000
1040000
1050000
1060000
1070000
1080000
1090000
1100000
1110000
1120000
1130000
1140000
1150000
1160000
1170000
1180000
1190000


1030.714719391999

In [13]:
dfPeopleIds.shape

(203181, 2)

In [14]:
#Tengo 23 ids que aparecen en mis películas pero no en mi base de datos de People. Bueno, asumo el ínfimo error.
dfPeople.shape

(203159, 13)

In [15]:
dfPeople.columns

Index(['adult', 'birthday', 'deathday', 'gender', 'imdb_id',
       'known_for_department', 'movie_credits_cast', 'movie_credits_crew',
       'name', 'place_of_birth', 'popularity', 'tv_credits_cast',
       'tv_credits_crew'],
      dtype='object')

In [16]:
#Compruebo que no se repite el identificador id de tmdb de las personas:
dfPeople.index.duplicated().sum()

0

In [17]:
process = psutil.Process(os.getpid())
print(process.memory_info().rss)

6261714944


###ANÁLISIS DE CAMPOS ESPECÍFICOS

####MI FUNCIÓN FLEXIBLE PARA TRATAR CUALQUIER COLUMNA CUYOS ELEMENTOS SON LISTAS DE DICCIONARIOS:

In [0]:
#columnName: columna de dfMovies a tratar. Sus elemeentos son listas de diccionarios.
#args: lista variable de claves con las que me voy a quedar de los diccionarios.
#Si un elemento es una lista vacía, no hay problema.
def columnIteratorPeople(columnName, *args):
    for id in dfPeople.index:
        for dict in dfPeople[dfPeople.index == id][columnName].iloc[0]:
            outputDict = {'people_id': id}
            for key in args:
                outputDict[key] = dict.get(key,None)
            yield outputDict

In [0]:
#Creo mi dataframe con la info de la columna correspondiente, compruebo que tiene el nº filas que ha de tener y, si es así, elimino la columna
#correspondiente en dfMovies. Haré todo esto con la siguiente función:
def columnProcessesPeople (columnName, *args):
    #I need to declare dfMovies as global as I will change it inside this function.
    global dfPeople
    
    dfOutput = pd.DataFrame()
    dfOutput = pd.DataFrame(columnIteratorPeople(columnName, *args))
    
    #Compruebo que tiene el nº de registros que ha de tener.
    dfCheck = pd.DataFrame(dfPeople[columnName].map(len).value_counts())
    dfCheck['calculation']=dfCheck[columnName]*dfCheck.index
    
    if dfOutput.shape[0] == dfCheck['calculation'].sum():
        dfPeople = dfPeople.drop(columnName, axis=1)
        return dfOutput
    else:
        return "Error in process"

####MOVIE_CREDITS_CAST

In [0]:
columnName = 'movie_credits_cast'

In [21]:
np.sort(dfPeople[columnName].map(len).unique())

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 171, 172, 174, 175, 176, 177, 178, 179, 180, 181, 18

In [22]:
dfPeople[columnName].map(len).value_counts()
#Observar cuántos hay con counts=0

0      90996
1      40328
2      15031
3       9082
4       6376
5       4710
6       3694
7       3048
8       2586
9       2119
10      1926
11      1616
12      1391
13      1313
14      1148
15      1079
16       887
18       834
17       811
19       735
20       687
21       606
22       595
23       549
24       524
25       479
26       469
28       395
29       383
27       377
       ...  
238        1
479        1
290        1
204        1
313        1
160        1
166        1
301        1
540        1
306        1
154        1
180        1
181        1
182        1
184        1
312        1
701        1
201        1
190        1
275        1
192        1
193        1
530        1
321        1
197        1
199        1
271        1
327        1
200        1
183        1
Name: movie_credits_cast, Length: 238, dtype: int64

In [23]:
#Veo un ejemplo:
dfPeople[dfPeople.index ==6885][columnName].iloc[0]

[{'adult': False,
  'backdrop_path': '/pexMctImHaIj8haojBDP5RxhM9X.jpg',
  'character': 'Aileen Wuornos',
  'credit_id': '52fe424bc3a36847f8013009',
  'genre_ids': [80, 18],
  'id': 504,
  'original_language': 'en',
  'original_title': 'Monster',
  'overview': "An emotionally scarred highway hooker shoots a sadistic trick who rapes her, and ultimately becomes America's first female serial killer.",
  'popularity': 7.218,
  'poster_path': '/aevmNtJCNG4ZlfEeEGZ79frMUes.jpg',
  'release_date': '2003-12-24',
  'title': 'Monster',
  'video': False,
  'vote_average': 7.1,
  'vote_count': 752},
 {'adult': False,
  'backdrop_path': '/bTyOP3kqN6a8m9lGh7NCruAnz5p.jpg',
  'character': 'Candy Kendall',
  'credit_id': '52fe430fc3a36847f80376d3',
  'genre_ids': [18],
  'id': 1715,
  'original_language': 'en',
  'original_title': 'The Cider House Rules',
  'overview': 'Homer is an orphan who was never adopted, becoming the favorite of orphanage director Dr. Larch. Dr. Larch imparts his full medical k

In [24]:
dfPeople[dfPeople.index ==6885][columnName].iloc[0][0]

{'adult': False,
 'backdrop_path': '/pexMctImHaIj8haojBDP5RxhM9X.jpg',
 'character': 'Aileen Wuornos',
 'credit_id': '52fe424bc3a36847f8013009',
 'genre_ids': [80, 18],
 'id': 504,
 'original_language': 'en',
 'original_title': 'Monster',
 'overview': "An emotionally scarred highway hooker shoots a sadistic trick who rapes her, and ultimately becomes America's first female serial killer.",
 'popularity': 7.218,
 'poster_path': '/aevmNtJCNG4ZlfEeEGZ79frMUes.jpg',
 'release_date': '2003-12-24',
 'title': 'Monster',
 'video': False,
 'vote_average': 7.1,
 'vote_count': 752}

In [26]:
#Veo en la documentación de themoviedb que estas keys son igules siempre.
dfPeople[dfPeople.index ==6885][columnName].iloc[0][0].keys()

dict_keys(['release_date', 'adult', 'vote_average', 'vote_count', 'video', 'title', 'popularity', 'genre_ids', 'original_language', 'character', 'original_title', 'poster_path', 'id', 'backdrop_path', 'overview', 'credit_id'])

In [27]:
#No voy a tomar la overview de la película en este caso
tic=timeit.default_timer()
dfPeopleMoviesCast = columnProcessesPeople (columnName, 'adult', 'character', 'genre_ids', 'id', 'original_language', 'original_title', 'popularity', 'release_date', 'title', 'vote_average', 'vote_count')
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

173.36695612099902

In [28]:
dfPeopleMoviesCast.shape

(996469, 12)

In [39]:
#Echamos un vistazo al dataframe:
dfPeopleMoviesCast.count()

adult                     0
character                 0
genre_ids                 0
id                        0
original_language         0
original_title            0
people_id            996469
popularity                0
release_date              0
title                     0
vote_average              0
vote_count                0
dtype: int64

In [0]:
#Miro mi ejemplo original:
dfPeopleMoviesCast[dfPeopleMoviesCast['people_id'] == 6885].iloc[0]

In [40]:
#Miro las columnas que quedan:
dfPeople.columns

Index(['adult', 'birthday', 'deathday', 'gender', 'imdb_id',
       'known_for_department', 'movie_credits_crew', 'name', 'place_of_birth',
       'popularity', 'tv_credits_cast', 'tv_credits_crew'],
      dtype='object')

In [0]:
dfPeople.shape

In [50]:
process = psutil.Process(os.getpid())
print(process.memory_info().rss)

6969212928


####MOVIE CREDITS CREW

In [0]:
columnName = 'movie_credits_crew'

In [59]:
np.sort(dfPeople[columnName].map(len).unique())

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 144,
       146, 147, 149, 155, 157, 158, 159, 161, 162, 163, 164, 165, 168,
       171, 173, 174, 176, 177, 183, 186, 196, 205, 207, 212, 213, 215,
       217, 218, 219, 229, 231, 234, 236, 248, 252, 256, 257, 26

In [60]:
dfPeople[columnName].map(len).value_counts()
#Observar cuántos hay con counts=0

0      89291
1      42005
2      19013
3      11473
4       7699
5       5582
6       4283
7       3316
8       2613
9       2167
10      1792
11      1514
12      1335
13      1073
14      1010
15       865
16       694
17       614
18       604
19       510
20       466
22       377
21       371
23       321
24       277
25       263
26       248
27       218
30       187
28       183
       ...  
309        1
140        1
157        1
620        1
335        1
339        1
155        1
212        1
213        1
215        1
280        1
343        1
217        1
158        1
219        1
334        1
229        1
147        1
173        1
231        1
159        1
234        1
144        1
491        1
108        1
205        1
236        1
397        1
492        1
556        1
Name: movie_credits_crew, Length: 198, dtype: int64

In [61]:
#Veo un ejemplo:
dfPeople[dfPeople.index ==6885][columnName].iloc[0]

[{'adult': False,
  'backdrop_path': '/pexMctImHaIj8haojBDP5RxhM9X.jpg',
  'credit_id': '52fe424bc3a36847f8013013',
  'department': 'Production',
  'genre_ids': [80, 18],
  'id': 504,
  'job': 'Producer',
  'original_language': 'en',
  'original_title': 'Monster',
  'overview': "An emotionally scarred highway hooker shoots a sadistic trick who rapes her, and ultimately becomes America's first female serial killer.",
  'popularity': 7.218,
  'poster_path': '/aevmNtJCNG4ZlfEeEGZ79frMUes.jpg',
  'release_date': '2003-12-24',
  'title': 'Monster',
  'video': False,
  'vote_average': 7.1,
  'vote_count': 752},
 {'adult': False,
  'backdrop_path': None,
  'credit_id': '52fe49e69251416c910bc7cf',
  'department': 'Production',
  'genre_ids': [],
  'id': 88021,
  'job': 'Producer',
  'original_language': 'en',
  'original_title': 'East of Havana',
  'overview': 'Documenting the hardships of three gifted Cuban rappers trying to survive in the harsh ghettos of Havana.',
  'popularity': 0.41200000

In [62]:
dfPeople[dfPeople.index ==6885][columnName].iloc[0][0]

{'adult': False,
 'backdrop_path': '/pexMctImHaIj8haojBDP5RxhM9X.jpg',
 'credit_id': '52fe424bc3a36847f8013013',
 'department': 'Production',
 'genre_ids': [80, 18],
 'id': 504,
 'job': 'Producer',
 'original_language': 'en',
 'original_title': 'Monster',
 'overview': "An emotionally scarred highway hooker shoots a sadistic trick who rapes her, and ultimately becomes America's first female serial killer.",
 'popularity': 7.218,
 'poster_path': '/aevmNtJCNG4ZlfEeEGZ79frMUes.jpg',
 'release_date': '2003-12-24',
 'title': 'Monster',
 'video': False,
 'vote_average': 7.1,
 'vote_count': 752}

In [63]:
dfPeople[dfPeople.index ==6885][columnName].iloc[0][0].keys()

dict_keys(['id', 'department', 'original_language', 'original_title', 'job', 'overview', 'genre_ids', 'video', 'credit_id', 'release_date', 'popularity', 'vote_average', 'vote_count', 'title', 'adult', 'backdrop_path', 'poster_path'])

In [65]:
tic=timeit.default_timer()
dfPeopleMoviesCrew = columnProcessesPeople (columnName, 'adult', 'department', 'genre_ids', 'id', 'job', 'original_language', 'original_title', 'popularity', 'release_date', 'title', 'vote_average', 'vote_count')
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

168.03668097199989

In [67]:
dfPeopleMoviesCrew.shape

(618620, 13)

In [68]:
dfPeopleMoviesCrew.count()

adult                618620
department           618620
genre_ids            618620
id                   618620
job                  618620
original_language    618620
original_title       618620
people_id            618620
popularity           618620
release_date         617815
title                618620
vote_average         618620
vote_count           618620
dtype: int64

In [69]:
dfPeopleMoviesCrew.tail()

,adult,department,genre_ids,id,job,original_language,original_title,people_id,popularity,release_date,title,vote_average,vote_count
618615,False,Writing,[],149576,Author,en,The Thought Exchange,1078101,0.127,2012-06-01,The Thought Exchange,9.0,2
618616,False,Sound,[],149576,Original Music Composer,en,The Thought Exchange,1078101,0.127,2012-06-01,The Thought Exchange,9.0,2
618617,False,Sound,"[35, 10749]",1812,Original Music Composer,en,Trick,1078101,3.711,1999-07-23,Trick,7.3,33
618618,False,Production,"[35, 27]",34223,Associate Producer,en,Vamp,1668101,2.444,1986-07-18,Vamp,5.8,51
618619,False,Writing,"[16, 35, 10751]",16508,Storyboard,en,Doug's 1st Movie,1466429,2.156,1999-03-26,Doug's 1st Movie,5.3,33


In [71]:
dfPeopleMoviesCrew[dfPeopleMoviesCrew['people_id'] == 6885]

,adult,department,genre_ids,id,job,original_language,original_title,people_id,popularity,release_date,title,vote_average,vote_count
60806,False,Production,"[80, 18]",504,Producer,en,Monster,6885,7.218,2003-12-24,Monster,7.1,752
60807,False,Production,[],88021,Producer,en,East of Havana,6885,0.412,2006-03-14,East of Havana,6.0,2
60808,False,Production,[35],340021,Producer,en,Everything Nice,6885,0.091,None,Everything Nice,10.0,1
60809,False,Production,"[28, 35, 80]",340022,Producer,en,Gringo,6885,14.822,2018-03-09,Gringo,5.9,190
60810,False,Production,[18],340027,Producer,en,Brain on Fire,6885,6.852,2017-06-06,Brain on Fire,7.2,348
60811,False,Production,"[18, 9648, 53]",182560,Producer,en,Dark Places,6885,8.574,2015-08-07,Dark Places,5.8,638
60812,False,Production,"[28, 9648, 53]",341013,Producer,en,Atomic Blonde,6885,17.113,2017-07-28,Atomic Blonde,6.2,2901
60813,False,Production,"[35, 18]",400579,Producer,en,Tully,6885,19.191,2018-05-04,Tully,6.9,218
60814,False,Production,"[28, 9648, 53]",522958,Producer,en,Atomic Blonde 2,6885,0.675,,Atomic Blonde 2,10.0,1
60815,False,Production,[18],14349,Producer,en,Sleepwalking,6885,3.290,2008-03-14,Sleepwalking,6.4,29


In [73]:
dfPeople.shape

(203159, 11)

In [74]:
dfPeople.columns

Index(['adult', 'birthday', 'deathday', 'gender', 'imdb_id',
       'known_for_department', 'name', 'place_of_birth', 'popularity',
       'tv_credits_cast', 'tv_credits_crew'],
      dtype='object')

In [75]:
process = psutil.Process(os.getpid())
print(process.memory_info().rss)

6949769216


####TV CREDITS CAST

In [0]:
columnName = 'tv_credits_cast'

In [77]:
np.sort(dfPeople[columnName].map(len).unique())

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98, 100, 101, 102, 103, 104,
       107, 108, 109, 110, 111, 112, 113, 114, 115, 117, 118, 123, 124,
       130, 140, 224])

In [78]:
dfPeople[columnName].map(len).value_counts()
#Observar cuántos hay con counts=0

0      162379
1       11970
2        5450
3        3510
4        2446
5        1870
6        1525
7        1332
8        1097
9         972
10        852
11        808
12        692
13        669
14        580
15        527
16        477
17        450
18        423
19        385
21        370
20        344
23        283
22        276
26        243
24        231
27        212
25        209
28        209
29        189
        ...  
103         3
102         3
117         2
112         2
118         2
111         2
109         2
101         2
87          2
93          2
97          1
110         1
123         1
82          1
130         1
115         1
114         1
113         1
86          1
88          1
91          1
224         1
108         1
107         1
104         1
140         1
94          1
96          1
100         1
124         1
Name: tv_credits_cast, Length: 120, dtype: int64

In [79]:
#Veo un ejemplo:
dfPeople[dfPeople.index ==6885][columnName].iloc[0]

[{'backdrop_path': '/7wri8hbMd1yZWIFubnZTe9ZJrKT.jpg',
  'character': '',
  'credit_id': '5253872d19c2957940217c5e',
  'episode_count': 2,
  'first_air_date': '2003-09-08',
  'genre_ids': [35],
  'id': 562,
  'name': 'The Ellen DeGeneres Show',
  'origin_country': ['US'],
  'original_language': 'en',
  'original_name': 'The Ellen DeGeneres Show',
  'overview': 'The Ellen DeGeneres Show, often shortened to Ellen, is an American television talk show hosted by comedian/actress Ellen DeGeneres.',
  'popularity': 5.128,
  'poster_path': '/AvfvsaqYt2oBYDC9c8N926mDHjl.jpg',
  'vote_average': 6.3,
  'vote_count': 35},
 {'backdrop_path': '/lt0GdLpUgFI09hkbTgvx8MvRS9m.jpg',
  'character': '',
  'credit_id': '525707e3760ee3776a0395b5',
  'episode_count': 3,
  'first_air_date': '2003-01-26',
  'genre_ids': [35, 10767],
  'id': 1489,
  'name': 'Jimmy Kimmel Live!',
  'origin_country': ['US'],
  'original_language': 'en',
  'original_name': 'Jimmy Kimmel Live!',
  'overview': 'Jimmy Kimmel Live! is 

In [80]:
dfPeople[dfPeople.index ==6885][columnName].iloc[0][0]

{'backdrop_path': '/7wri8hbMd1yZWIFubnZTe9ZJrKT.jpg',
 'character': '',
 'credit_id': '5253872d19c2957940217c5e',
 'episode_count': 2,
 'first_air_date': '2003-09-08',
 'genre_ids': [35],
 'id': 562,
 'name': 'The Ellen DeGeneres Show',
 'origin_country': ['US'],
 'original_language': 'en',
 'original_name': 'The Ellen DeGeneres Show',
 'overview': 'The Ellen DeGeneres Show, often shortened to Ellen, is an American television talk show hosted by comedian/actress Ellen DeGeneres.',
 'popularity': 5.128,
 'poster_path': '/AvfvsaqYt2oBYDC9c8N926mDHjl.jpg',
 'vote_average': 6.3,
 'vote_count': 35}

In [81]:
dfPeople[dfPeople.index ==6885][columnName].iloc[0][0].keys()

dict_keys(['origin_country', 'original_name', 'genre_ids', 'vote_count', 'backdrop_path', 'name', 'first_air_date', 'original_language', 'popularity', 'character', 'episode_count', 'id', 'credit_id', 'vote_average', 'overview', 'poster_path'])

In [82]:
tic=timeit.default_timer()
dfPeopleTvCast = columnProcessesPeople (columnName, 'character', 'episode_count', 'first_air_date', 'genre_ids', 'id', 'name', 'origin_country', 'original_language', 'original_name', 'popularity', 'vote_average', 'vote_count')
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

160.8628852460024

In [83]:
dfPeopleTvCast.shape

(325921, 13)

In [84]:
dfPeopleTvCast.count()

character            325921
episode_count        325920
first_air_date       325887
genre_ids            325921
id                   325921
name                 325920
origin_country       325920
original_language    325921
original_name        325920
people_id            325921
popularity           325921
vote_average         325921
vote_count           325921
dtype: int64

In [85]:
dfPeopleTvCast.tail()

,character,episode_count,first_air_date,genre_ids,id,name,origin_country,original_language,original_name,people_id,popularity,vote_average,vote_count
325916,"Penelope ""Penny"" Halliwell",15.0,1998-10-07,"[35, 18, 9648]",1981,Charmed,[US],en,Charmed,27568,16.368,6.8,240
325917,Bus Driver,1.0,2016-09-21,[35],67040,Speechless,[US],en,Speechless,165359,9.482,7.5,44
325918,,1.0,1980-12-11,"[18, 10759]",734,"Magnum, P.I.",[US],en,"Magnum, P.I.",106811,25.180,7.1,75
325919,,1.0,1973-10-24,"[18, 10759]",3572,Kojak,[US],en,Kojak,106811,2.597,7.3,31
325920,,1.0,1972-09-13,"[10759, 10765]",11398,Search,[US],en,Search,106811,1.317,8.9,4


In [86]:
dfPeopleTvCast[dfPeopleTvCast['people_id'] == 6885]

,character,episode_count,first_air_date,genre_ids,id,name,origin_country,original_language,original_name,people_id,popularity,vote_average,vote_count
32136,,2.0,2003-09-08,[35],562,The Ellen DeGeneres Show,[US],en,The Ellen DeGeneres Show,6885,5.128,6.3,35
32137,,3.0,2003-01-26,"[35, 10767]",1489,Jimmy Kimmel Live!,[US],en,Jimmy Kimmel Live!,6885,2.836,5.9,34
32138,,3.0,1983-04-04,[],1900,Live with Regis and Kathie Lee,[US],en,Live with Regis and Kathie Lee,6885,2.536,4.9,4
32139,,1.0,1997-08-11,[10767],2221,The View,[US],en,The View,6885,17.604,3.7,23
32140,,2.0,1996-07-22,"[35, 10763]",2224,The Daily Show with Trevor Noah,[US],en,The Daily Show with Trevor Noah,6885,43.025,6.7,195
32141,,1.0,1992-05-25,"[35, 10767]",2518,The Tonight Show with Jay Leno,[US],en,The Tonight Show with Jay Leno,6885,14.895,5.0,35
32142,,3.0,1993-09-13,"[35, 10767]",4573,Late Night with Conan O'Brien,[US],en,Late Night with Conan O'Brien,6885,17.536,7.4,52
32143,,5.0,2003-11-02,[35],4589,Arrested Development,[US],en,Arrested Development,6885,14.924,8.2,418
32144,Rita,26.0,2003-11-02,[35],4589,Arrested Development,[US],en,Arrested Development,6885,14.924,8.2,418
32145,,1.0,1994-06-01,[10767],217,Inside the Actors Studio,[],en,Inside the Actors Studio,6885,0.459,7.6,20


In [87]:
dfPeople.shape

(203159, 10)

In [88]:
dfPeople.columns

Index(['adult', 'birthday', 'deathday', 'gender', 'imdb_id',
       'known_for_department', 'name', 'place_of_birth', 'popularity',
       'tv_credits_crew'],
      dtype='object')

In [89]:
process = psutil.Process(os.getpid())
print(process.memory_info().rss)

6947229696


#### TV CREDITS CREW

In [0]:
columnName = 'tv_credits_crew'

In [91]:
np.sort(dfPeople[columnName].map(len).unique())

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  55,  59,  60,  63,  64,  70,  89, 131, 132])

In [92]:
dfPeople[columnName].map(len).value_counts()
#Observar cuántos hay con counts=0

0      190368
1        7280
2        2109
3         980
4         503
5         355
6         234
7         208
8         147
9         114
10        112
11        100
12         68
13         56
14         53
15         44
17         37
18         32
16         31
22         22
23         22
19         21
25         21
20         20
21         18
30         14
24         14
32         14
26         13
29         12
        ...  
27         10
35          9
34          8
37          8
31          7
46          6
42          5
43          5
44          4
39          4
41          3
64          3
52          3
36          3
53          3
63          2
60          2
59          2
45          2
50          2
49          2
48          2
51          1
55          1
132         1
131         1
47          1
70          1
40          1
89          1
Name: tv_credits_crew, Length: 63, dtype: int64

In [93]:
#Veo un ejemplo:
dfPeople[dfPeople.index ==6885][columnName].iloc[0]

[{'backdrop_path': '/1Dxe2ZXesgLsbpJI5m6PyKIVk93.jpg',
  'credit_id': '56e1f15e9251417896000357',
  'department': 'Production',
  'episode_count': 13,
  'first_air_date': '2017-04-21',
  'genre_ids': [35],
  'id': 65856,
  'job': 'Executive Producer',
  'name': 'Girlboss',
  'origin_country': ['US'],
  'original_language': 'en',
  'original_name': 'Girlboss',
  'overview': 'Sophia is a rebellious, broke anarchist who refuses to grow up. She stumbles upon her passion of selling vintage clothes online and becomes an unlikely businesswoman. As she builds her retail fashion empire, she realizes the value and the difficulty of being the boss of her own life.',
  'popularity': 5.702,
  'poster_path': '/w60XVdh93UU9k7yyXFItnZkVA5y.jpg',
  'vote_average': 6.8,
  'vote_count': 33},
 {'backdrop_path': '/a906PH7CDmSOdS7kmnAgdWk5mhv.jpg',
  'credit_id': '58ba5a70925141607b019714',
  'department': 'Production',
  'episode_count': 10,
  'first_air_date': '2017-10-13',
  'genre_ids': [80, 18],
  'id'

In [94]:
dfPeople[dfPeople.index ==6885][columnName].iloc[0][0]

{'backdrop_path': '/1Dxe2ZXesgLsbpJI5m6PyKIVk93.jpg',
 'credit_id': '56e1f15e9251417896000357',
 'department': 'Production',
 'episode_count': 13,
 'first_air_date': '2017-04-21',
 'genre_ids': [35],
 'id': 65856,
 'job': 'Executive Producer',
 'name': 'Girlboss',
 'origin_country': ['US'],
 'original_language': 'en',
 'original_name': 'Girlboss',
 'overview': 'Sophia is a rebellious, broke anarchist who refuses to grow up. She stumbles upon her passion of selling vintage clothes online and becomes an unlikely businesswoman. As she builds her retail fashion empire, she realizes the value and the difficulty of being the boss of her own life.',
 'popularity': 5.702,
 'poster_path': '/w60XVdh93UU9k7yyXFItnZkVA5y.jpg',
 'vote_average': 6.8,
 'vote_count': 33}

In [95]:
dfPeople[dfPeople.index ==6885][columnName].iloc[0][0].keys()

dict_keys(['id', 'department', 'original_language', 'episode_count', 'job', 'overview', 'origin_country', 'original_name', 'vote_count', 'name', 'popularity', 'credit_id', 'backdrop_path', 'first_air_date', 'vote_average', 'genre_ids', 'poster_path'])

In [96]:
tic=timeit.default_timer()
dfPeopleTvCrew = columnProcessesPeople (columnName, 'department', 'episode_count', 'first_air_date', 'genre_ids', 'id', 'job', 'name', 'origin_country', 'original_language', 'original_name', 'popularity', 'vote_average', 'vote_count')
toc=timeit.default_timer()
toc - tic #elapsed time in seconds

157.40268017500057

In [97]:
dfPeopleTvCrew.shape

(40362, 14)

In [98]:
dfPeopleTvCrew.count()

department           40362
episode_count        40362
first_air_date       40356
genre_ids            40362
id                   40362
job                  40362
name                 40362
origin_country       40362
original_language    40362
original_name        40362
people_id            40362
popularity           40362
vote_average         40362
vote_count           40362
dtype: int64

In [99]:
dfPeopleTvCrew.tail()

,department,episode_count,first_air_date,genre_ids,id,job,name,origin_country,original_language,original_name,people_id,popularity,vote_average,vote_count
40357,Directing,12,2014-04-11,"[16, 10759, 10765]",60831,Director,Knights of Sidonia,[JP],ja,シドニアの騎士,237072,3.529,7.7,20
40358,Editing,2,2016-09-20,"[35, 18]",67136,Assistant Editor,This Is Us,[US],en,This Is Us,1730317,24.050,8.1,264
40359,Costume & Make-Up,1,2011-04-17,"[18, 10759, 10765]",1399,Makeup Effects Designer,Game of Thrones,[US],en,Game of Thrones,101608,56.630,8.2,4792
40360,Production,6,2017-04-04,[10765],70690,Casting,Dimension 404,[US],en,Dimension 404,1021206,5.911,6.9,22
40361,Crew,1,2009-09-22,[18],1435,Transportation Coordinator,The Good Wife,[US],en,The Good Wife,1410317,26.867,7.4,170


In [100]:
dfPeopleTvCrew[dfPeopleTvCrew['people_id'] == 6885]

,department,episode_count,first_air_date,genre_ids,id,job,name,origin_country,original_language,original_name,people_id,popularity,vote_average,vote_count
4028,Production,13,2017-04-21,[35],65856,Executive Producer,Girlboss,[US],en,Girlboss,6885,5.702,6.8,33
4029,Production,10,2017-10-13,"[80, 18]",67744,Executive Producer,Mindhunter,[US],en,Mindhunter,6885,45.600,7.4,443
4030,Production,1,2019-01-01,[18],81508,Producer,A Life in Men,[],en,A Life in Men,6885,0.008,0.0,0


In [101]:
dfPeople.shape

(203159, 9)

In [102]:
dfPeople.columns

Index(['adult', 'birthday', 'deathday', 'gender', 'imdb_id',
       'known_for_department', 'name', 'place_of_birth', 'popularity'],
      dtype='object')

In [103]:
process = psutil.Process(os.getpid())
print(process.memory_info().rss)

6943084544


In [0]:
#Generación de todas las tablas de una tacada: 
#This module took about XXmin to be executed in Google Colab.
tic=timeit.default_timer()

dfPeopleMoviesCast = columnProcessesPeople (columnName, 'adult', 'character', 'genre_ids', 'id', 'original_language', 'original_title', 'popularity', 'release_date', 'title', 'vote_average', 'vote_count')
print("dfPeopleMoviesCast generated")
dfPeopleMoviesCrew = columnProcessesPeople (columnName, 'adult', 'department', 'genre_ids', 'id', 'job', 'original_language', 'original_title', 'popularity', 'release_date', 'title', 'vote_average', 'vote_count')
print("dfPeopleMoviesCrew generated")
dfPeopleTvCast = columnProcessesPeople (columnName, 'character', 'episode_count', 'first_air_date', 'genre_ids', 'id', 'name', 'origin_country', 'original_language', 'original_name', 'popularity', 'vote_average', 'vote_count')
print("dfPeopleTvCast generated")
dfPeopleTvCrew = columnProcessesPeople (columnName, 'department', 'episode_count', 'first_air_date', 'genre_ids', 'id', 'job', 'name', 'origin_country', 'original_language', 'original_name', 'popularity', 'vote_average', 'vote_count')
print("dfPeopleTvCrew generated")

toc=timeit.default_timer()
toc - tic #elapsed time in seconds